# **Planet: Understanding the Amazon from Space**

# **Neural Network, Image Recognition & Object Detection**

**Importing the libraries**

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import gc
import numpy as np

from keras import backend as K
from sklearn.metrics import fbeta_score
from keras.layers import Conv2D, Dense, LSTM, Flatten, MaxPooling2D, BatchNormalization, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import fbeta_score
from tqdm import tqdm
from sklearn.utils import shuffle

import cv2
from PIL import Image
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential 

import seaborn as sns
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History

**Loading the datasets**

In [2]:
#reading the labels

train_label = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
sam_sub = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')
train_label.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [3]:
#onehot=OneHotEncoder()

encoder = LabelEncoder()

In [4]:
#encoder.fit(train_label['tags'])

label_maps = pd.DataFrame()
label_maps['tags'] = ['agriculture', 'artisinal_mine', 'bare_ground','blooming','blow_down','clear','cloudy','conventional_mine','cultivation','habitation','haze', 'partly_cloudy','primary','road','selective_logging','slash_burn','water']
label_maps['map'] = encoder.fit_transform(label_maps['tags'])
label_maps

,tags,map
0,agriculture,0
1,artisinal_mine,1
2,bare_ground,2
3,blooming,3
4,blow_down,4
5,clear,5
6,cloudy,6
7,conventional_mine,7
8,cultivation,8
9,habitation,9


In [5]:
#defining a dict of encoded labels

label_map = {'agriculture': 0,
 'artisinal_mine': 1,
 'bare_ground': 2,
 'blooming': 3,
 'blow_down': 4,
 'clear': 5,
 'cloudy': 6,
 'conventional_mine': 7,
 'cultivation': 8,
 'habitation': 9,
 'haze': 10,
 'partly_cloudy': 11,
 'primary': 12,
 'road': 13,
 'selective_logging': 14,
 'slash_burn': 15,
 'water': 16}


In [6]:
#loading the traing_images

X = []
Y = []
train_label = shuffle(train_label,random_state=0)

In [7]:
for image_name, tags in tqdm(train_label.values, miniters=400):
    arr = cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(image_name), cv2.IMREAD_UNCHANGED)
    targets = np.zeros(17)
    for t in tags.split(' '):
      targets[label_map[t]] = 1 
    arr = cv2.resize(arr, (64, 64))
    X.append(arr)
    Y.append(targets)

100%|██████████| 40479/40479 [04:28<00:00, 150.85it/s]


In [8]:
X = np.array(X, np.float16)/255.0

In [9]:
#splitting

X = np.array(X)
Y = np.array(Y)
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, shuffle = True, random_state = 1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(32383, 64, 64, 3) (32383, 17) (8096, 64, 64, 3) (8096, 17)


In [10]:
del X, Y
gc.collect()

126

In [11]:
gc.collect()

21

In [12]:
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [13]:
from tensorflow.keras.optimizers import Adam

**Define the model**

In [14]:
model = keras.Sequential()
model.add(Conv2D(64, 5, 2, activation = "relu", input_shape = (64, 64, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(128, 5, 2, activation = "relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dense(17, activation = "sigmoid"))
model.compile(loss = "binary_crossentropy", optimizer = Adam(), metrics = [fbeta])
model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 45, batch_size = 64)

Epoch 1/45
506/506 [==============================] - 14s 9ms/step - loss: 0.2243 - fbeta: 0.6421 - val_loss: 0.2006 - val_fbeta: 0.6577
Epoch 2/45
506/506 [==============================] - 3s 6ms/step - loss: 0.1992 - fbeta: 0.6831 - val_loss: 0.1925 - val_fbeta: 0.7185
Epoch 3/45
506/506 [==============================] - 3s 6ms/step - loss: 0.1867 - fbeta: 0.7080 - val_loss: 0.1781 - val_fbeta: 0.7352
Epoch 4/45
506/506 [==============================] - 3s 6ms/step - loss: 0.1726 - fbeta: 0.7323 - val_loss: 0.1639 - val_fbeta: 0.7566
Epoch 5/45
506/506 [==============================] - 3s 6ms/step - loss: 0.1631 - fbeta: 0.7498 - val_loss: 0.1609 - val_fbeta: 0.7556
Epoch 6/45
506/506 [==============================] - 3s 6ms/step - loss: 0.1574 - fbeta: 0.7589 - val_loss: 0.1600 - val_fbeta: 0.7678
Epoch 7/45
506/506 [==============================] - 3s 6ms/step - loss: 0.1528 - fbeta: 0.7654 - val_loss: 0.1473 - val_fbeta: 0.7724
Epoch 8/45
506/506 [===========================

In [15]:
gc.collect()

1765

In [16]:
#with tpu_strategy.scope():

test_loss, test_accuracy = model.evaluate(x_val, y_val)
print('Test loss: {}'.format(test_loss))
print('Test accuracy: {}'.format(test_accuracy))

253/253 [==============================] - 1s 3ms/step - loss: 0.1889 - fbeta: 0.8108
Test loss: 0.18889395892620087
Test accuracy: 0.8108479976654053


In [17]:
gc.collect()

788

In [18]:
del x_train, y_train, x_val, y_val
gc.collect()

21

In [19]:
#dividing my test_labels into two part for test-jpg and test-jpg-additional
test = sam_sub[0 : 40669]
files = sam_sub[40669 : ]

In [20]:
#with tpu_strategy.scope():  

test_img = []

for image_name, tags in tqdm(test.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/planet/planet/test-jpg/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

for image_name, tags in tqdm(files.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/test-jpg-additional/test-jpg-additional/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

test_img = np.array(test_img, np.float16)/255.0

100%|██████████| 20522/20522 [02:15<00:00, 151.74it/s]


In [21]:
gc.collect()

63

In [22]:
#with tpu_strategy.scope():

yres = []
predictions = model.predict(test_img, batch_size = 64, verbose = 2)
yres.append(predictions)

957/957 - 2s


In [23]:
gc.collect()

781

In [24]:
#converting my encoded labels back to it original form

sub = np.array(yres[0])
for i in range (1, len(yres)):
    sub += np.array(yres[i])
sub = pd.DataFrame(sub, columns = label_map)

In [25]:
sub

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,6.079806e-04,6.474367e-12,6.717495e-05,1.773575e-02,7.155618e-05,9.998248e-01,1.816512e-06,1.789870e-13,2.356217e-04,5.046153e-04,1.635612e-04,3.168952e-06,0.999998,0.001842,9.562597e-04,6.021023e-12,5.956087e-03
1,4.678944e-02,2.995439e-13,8.554623e-06,9.127068e-04,1.163081e-03,9.791436e-01,5.117465e-09,6.041782e-14,2.658501e-03,2.709416e-03,3.938048e-06,9.108572e-02,1.000000,0.068570,3.306898e-03,1.793497e-08,1.627048e-02
2,7.686615e-03,1.468751e-19,1.346648e-12,2.801653e-13,2.003494e-15,3.894727e-10,6.154454e-09,2.138062e-10,2.430274e-05,3.731662e-09,1.159789e-09,1.000000e+00,1.000000,0.000035,9.742698e-11,2.905090e-13,8.205881e-07
3,4.985751e-02,1.384151e-10,2.237543e-04,3.372225e-02,4.497180e-03,5.810994e-01,4.748921e-07,1.480399e-10,1.205053e-02,9.471998e-04,1.940431e-04,4.293733e-01,0.999998,0.050536,4.491268e-03,2.472032e-07,2.194459e-02
4,4.616207e-05,2.449752e-18,6.998695e-09,7.623086e-15,7.381484e-21,1.187862e-09,4.967587e-05,2.588602e-16,2.226848e-10,3.904257e-09,1.701862e-06,9.997093e-01,0.994975,0.002624,1.215637e-12,3.854685e-14,2.806645e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61186,3.519040e-07,5.002523e-17,1.758182e-13,7.811859e-19,3.655650e-27,6.274769e-06,9.931329e-01,5.282598e-21,7.405851e-10,1.917516e-05,1.781940e-04,3.798243e-03,0.014600,0.000424,3.896679e-19,3.296821e-23,1.300437e-02
61187,1.942565e-02,2.485626e-11,3.058632e-04,4.020042e-12,1.470043e-16,1.000000e+00,8.258623e-18,1.084238e-07,3.381441e-01,7.349682e-04,7.054944e-13,3.797730e-12,0.999993,0.000893,1.287340e-13,5.688529e-07,9.997994e-01
61188,6.046855e-02,5.229619e-12,4.650133e-04,3.830188e-02,1.378356e-02,9.995938e-01,1.610991e-07,4.545773e-12,4.621901e-02,4.252904e-03,9.084596e-05,1.618130e-04,0.999998,0.236670,1.716364e-02,3.312732e-06,6.765600e-02
61189,1.389206e-03,2.438409e-15,6.196442e-10,9.935658e-17,3.639938e-20,1.726933e-03,9.434649e-01,1.282464e-15,5.233046e-04,1.298685e-04,7.881738e-02,5.480288e-06,0.050786,0.000450,3.034259e-13,2.507864e-19,4.773716e-03


In [26]:
preds = []
for i in tqdm(range(sub.shape[0]), miniters=1000):
    a = sub.loc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a= a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

sam_sub['tags'] = preds
sam_sub.to_csv('submission.csv', index=False)

100%|██████████| 61191/61191 [01:59<00:00, 510.76it/s]
